In [1]:
import importlib
import commai_spd_chl_keras_datagen 
importlib.reload(commai_spd_chl_keras_datagen)

import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [10]:
seq = Sequential()

seq.add(Conv2D(filters=64,kernel_initializer='he_normal', kernel_size=(4,4),padding='same',input_shape=(480,640,3)))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())


seq.add(Conv2D(filters=96, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=96, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(2*96,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense(1,kernel_initializer='he_normal'))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 480, 640, 64)      3136      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 120, 160, 64)      0         
_________________________________________________________________
activation_32 (Activation)   (None, 120, 160, 64)      0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 120, 160, 64)      256       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 120, 160, 64)      65600     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 30, 40, 64)        0         
_________________________________________________________________
activation_33 (Activation)   (None, 30, 40, 64)       

In [12]:
commai_speed_chl_gen(v="data_new/train.mp4",db_name="/mnt/imgs",txt="data_new/train.txt").build_video_folder()

  0%|          | 15/53831 [00:00<06:14, 143.72it/s]

(480, 640)
(32, 3, 480, 640)
(32, 3, 1)
making dir:, /mnt/imgs
building video Database


100%|██████████| 53831/53831 [11:50<00:00, 75.79it/s] 


In [4]:
batch_size=45
n_e=120
# total new frames:53830
n_frames_train=45600
n_frames_test=4000
n_frames_valid=4000

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="optical_flow_dense",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size,
                                                  v="data_new/train.mp4",txt="data_new/train.txt",
                                                  db_name="/mnt/imgs").get_indexes()

load=np.load(open("idxes_45bs_opflowdense_KITTI.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=train_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
test_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=test_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=valid_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)

#np.savez_compressed(open("idxes_45bs_opflowdense_KITTI.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
train_samples: 22800 
test_samples:2000 
valid_samples: 2000
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)


In [6]:
a=[]
import time
t=time.time()
for i in test_gen:
    a.append(i)
    break
print(time.time()-t) # fuck me 

4.71239161491394


In [ ]:
ie=0
filepath="STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=25,max_queue_size=100,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/120
506/506 [==============================] - ETA: 0s - loss: 135.0914WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-01-66.80-135.09.hdf5
506/506 [==============================] - 797s 2s/step - loss: 135.0914 - val_loss: 66.8040
Epoch 2/120
506/506 [==============================] - ETA: 0s - loss: 90.2646WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-02-65.56-90.26.hdf5
506/506 [==============================] - 783s 2s/step - loss: 90.2646 - val_loss: 65.5637


Epoch 00008: saving model to STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-08-23.55-27.39.hdf5
506/506 [==============================] - 786s 2s/step - loss: 27.3915 - val_loss: 23.5547
Epoch 9/120
506/506 [==============================] - ETA: 0s - loss: 24.9499WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-09-23.91-24.95.hdf5
506/506 [==============================] - 798s 2s/step - loss: 24.9499 - val_loss: 23.9093
Epoch 10/120
506/506 [==============================] - ETA: 0s - loss: 21.5321WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_no

In [5]:
#44 Epoches
seq=load_model("STACKED_Wide-CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-44-14.04-8.10.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 20,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
45/44 - 35s - loss: 13.8705


13.87048625946045